In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import sympy as sy
import sparse as sp
import math

In [ ]:
from qgs.params.params import QgParams
from qgs.functions.tendencies import create_tendencies

In [ ]:
model_parameters = QgParams(dynamic_T=True)

In [ ]:
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Mode truncation at the wavenumber 2 in the x and at the 
# wavenumber 4 in the y spatial coordinates for the ocean
model_parameters.set_oceanic_basin_fourier_modes(2, 4, mode="symbolic")

In [ ]:
model_parameters.symbolic_params

In [ ]:
from qgs.inner_products.symbolic import AtmosphericSymbolicInnerProducts, OceanicSymbolicInnerProducts
from qgs.tensors.qgtensor import QgsTensor, QgsTensorDynamicT, QgsTensorT4
from qgs.tensors.symbolic_qgtensor import SymbolicTensorLinear, SymbolicTensorDynamicT
from qgs.tensors.symbolic_qgtensor import _shift_dict_keys, _add_to_dict

In [ ]:
%%time
# Takes ~5 mins
# atm_ip = AtmosphericSymbolicInnerProducts(model_parameters)
# ocn_ip_stored = OceanicSymbolicInnerProducts(model_parameters, stored=True, return_symbolic=True)
ocn_ip = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
%%time
# Takes ~5 mins
# atm_ip_stored = AtmosphericSymbolicInnerProducts(model_parameters, return_symbolic=True)
atm_ip = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
# atm_ip_stored.save_to_file('temp_atm_sym_de.ip')
# ocn_ip_stored.save_to_file('temp_ocn_sym_de.ip')

In [ ]:
atm_loaded = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)
ocn_loaded = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
atm_loaded.load_from_file('temp_atm_sym_de.ip')
ocn_loaded.load_from_file('temp_ocn_sym_de.ip')

## Testing the symbolic_qgtensor

In [ ]:
ten = SymbolicTensorDynamicT(params=model_parameters, atmospheric_inner_products=atm_loaded, oceanic_inner_products=ocn_loaded)

In [ ]:
_, _, ten_num = create_tendencies(model_parameters, return_qgtensor=True)

In [ ]:
# ten.test_tensor_numerically(tol=1e-9)

In [ ]:
# atm_num = AtmosphericSymbolicInnerProducts(model_parameters, stored=True)
# ocn_num = OceanicSymbolicInnerProducts(model_parameters, stored=True)

# ocn_num_ns = OceanicSymbolicInnerProducts(model_parameters, stored=False)

In [ ]:
from qgs.functions.symbolic_output import create_symbolic_equations, translate_equations, print_equations, equation_as_function

In [ ]:
%%time
funcs = create_symbolic_equations(model_parameters, atm_loaded, ocn_loaded)

In [ ]:
model_parameters._set_symbolic_parameters()

In [ ]:
%%time
eqs = print_equations(funcs[0], model_parameters, subs=True, return_equations=True)

In [ ]:
%%time
fx = equation_as_function(funcs[0], model_parameters, subs=True, language='python', string_output=True)

In [ ]:
for i in fx:
    print(i)

In [ ]:
from numba import njit

In [ ]:
@njit
def f(t, U):
    #Tendency function of the qgs model
    U_out = np.empty_like(U)
    U_out[0] = -0.05*U[0] + 0.05*U[11] + 0.168986428105931*U[22] + 0.251005505082614*U[24]
    U_out[1] = -0.980418808722262*U[0]*U[2] - 0.980418808722262*U[11]*U[13] + 0.05*U[12] - 1.56867009395562*U[14]*U[16] - 1.50055762081784*U[15]*U[18] + 1.50055762081784*U[16]*U[17] - 0.05*U[1] - 0.0112217054051163*U[21] + 0.120745727252785*U[2] - 1.56867009395562*U[3]*U[5] - 1.50055762081784*U[4]*U[7] + 1.50055762081784*U[5]*U[6]
    U_out[2] = 0.980418808722262*U[0]*U[1] + 0.980418808722262*U[11]*U[12] + 0.05*U[13] + 1.56867009395562*U[14]*U[15] + 1.50055762081784*U[15]*U[17] + 1.50055762081784*U[16]*U[18] - 0.120745727252785*U[1] + 0.05*U[25] - 0.05*U[2] + 1.56867009395562*U[3]*U[4] + 1.50055762081784*U[4]*U[6] + 1.50055762081784*U[5]*U[7]
    U_out[3] = 1.87265793760678*U[12]*U[16] - 1.87265793760678*U[13]*U[15] + 0.05*U[14] + 3.74531587521356*U[17]*U[20] - 3.74531587521356*U[18]*U[19] + 1.87265793760678*U[1]*U[5] - 0.00679432430697251*U[21] + 0.0810088839426413*U[23] - 1.87265793760678*U[2]*U[4] - 0.05*U[3] + 3.74531587521356*U[6]*U[9] - 3.74531587521356*U[7]*U[8]
    U_out[4] = -1.02902937637678*U[0]*U[5] - 1.02902937637678*U[11]*U[16] + 1.73752196836555*U[12]*U[18] + 0.574852231579352*U[13]*U[14] - 1.73752196836555*U[13]*U[17] + 0.05*U[15] + 1.73752196836555*U[1]*U[7] - 0.0164935614721478*U[22] + 0.574852231579352*U[2]*U[3] - 1.73752196836555*U[2]*U[6] - 0.05*U[4] + 0.0570836566449895*U[5]
    U_out[5] = 1.02902937637678*U[0]*U[4] + 1.02902937637678*U[11]*U[15] - 0.574852231579352*U[12]*U[14] - 1.73752196836555*U[12]*U[17] - 1.73752196836555*U[13]*U[18] + 0.05*U[16] - 0.574852231579352*U[1]*U[3] - 1.73752196836555*U[1]*U[6] + 0.05*U[26] - 1.73752196836555*U[2]*U[7] - 0.0570836566449895*U[4] - 0.05*U[5]
    U_out[6] = -2.71889339738442*U[0]*U[7] - 2.71889339738442*U[11]*U[18] + 0.753865979381443*U[12]*U[16] + 0.753865979381443*U[13]*U[15] - 4.35022943581506*U[14]*U[20] + 0.05*U[17] + 0.753865979381443*U[1]*U[5] - 0.000777999678859866*U[21] + 0.753865979381443*U[2]*U[4] - 4.35022943581506*U[3]*U[9] - 0.05*U[6] + 0.0837128882242243*U[7]
    U_out[7] = 2.71889339738442*U[0]*U[6] + 2.71889339738442*U[11]*U[17] - 0.753865979381443*U[12]*U[15] + 0.753865979381443*U[13]*U[16] + 4.35022943581506*U[14]*U[19] + 0.05*U[18] - 0.753865979381443*U[1]*U[4] + 0.753865979381443*U[2]*U[5] + 4.35022943581506*U[3]*U[8] - 0.0837128882242243*U[6] - 0.05*U[7]
    U_out[8] = -2.26482546109568*U[0]*U[9] - 2.26482546109568*U[11]*U[20] - 1.7450294536311*U[14]*U[18] + 0.05*U[19] - 0.0017443933973331*U[22] - 1.7450294536311*U[3]*U[7] - 0.05*U[8] + 0.0603728636263923*U[9]
    U_out[9] = 2.26482546109568*U[0]*U[8] + 2.26482546109568*U[11]*U[19] + 1.7450294536311*U[14]*U[17] + 0.05*U[20] + 1.7450294536311*U[3]*U[6] - 0.0603728636263923*U[8] - 0.05*U[9]
    U_out[10] = -0.00055453242558364*U[10]**4 - 0.0193798449612403*U[10] - 0.00576517534924902*U[21] - 0.0127293180382657*U[23] + 1.73291382994887e-5*U[29]**4 + 5.6185881728191e-5*U[29]**3*U[30] + 1.87286272427303e-5*U[29]**3*U[32] + 0.00968992248062015*U[29] + 0.00785435532111145*U[30] + 0.00261811844037048*U[32] + 0.000515537550678319
    U_out[11] = 0.00454545454545455*U[0] - 0.00201648154757687*U[10]**3*U[11] - 0.0239816772374912*U[11] - 1.41868025576271*U[12]*U[2] + 1.41868025576271*U[13]*U[1] - 1.13494420461017*U[15]*U[5] + 1.13494420461017*U[16]*U[4] - 2.83736051152543*U[17]*U[7] + 2.83736051152543*U[18]*U[6] - 2.26988840922034*U[19]*U[9] + 2.26988840922034*U[20]*U[8] - 0.0153624025550847*U[22] - 0.0228186822802377*U[24] + 4.81568702750899e-5*U[29]**3*U[31] + 1.9262748110036e-5*U[29]**3*U[33] + 0.00673196110231097*U[31] + 0.00269278444092439*U[33] + 0.000468670500616653
    U_out[12] = -1.43757363347933*U[0]*U[13] - 0.00174793514762377*U[10]**3*U[12] + 1.02191932371371*U[11]*U[2] - 0.030110200915083*U[12] + 0.0255954299692664*U[13] + 1.63507091794193*U[14]*U[5] + 1.21855791962175*U[15]*U[7] - 2.30011781356693*U[16]*U[3] - 1.21855791962175*U[16]*U[6] + 1.85472813238771*U[17]*U[5] - 1.85472813238771*U[18]*U[4] + 0.0105988967691095*U[1] + 0.00237875394324372*U[21] - 2.31827099123654e-5*U[29]**3*U[30] - 0.00324076503486835*U[30]
    U_out[13] = 1.43757363347933*U[0]*U[12] - 0.00174793514762377*U[10]**3*U[13] - 1.02191932371371*U[11]*U[1] - 0.0255954299692664*U[12] - 0.030110200915083*U[13] - 1.63507091794193*U[14]*U[4] + 2.30011781356693*U[15]*U[3] - 1.21855791962175*U[15]*U[6] - 1.21855791962175*U[16]*U[7] + 1.85472813238771*U[17]*U[4] + 1.85472813238771*U[18]*U[5] - 0.0105988967691095*U[25] + 5.46229733632427e-5*U[29]**3*U[34] + 0.0105988967691095*U[2] + 0.00763587271916482*U[34]
    U_out[14] = -0.0015843783588104*U[10]**3*U[14] - 1.24843862507119*U[12]*U[5] + 1.24843862507119*U[13]*U[4] - 0.0338427464008859*U[14] - 2.3185288751322*U[15]*U[2] + 2.3185288751322*U[16]*U[1] - 2.49687725014238*U[17]*U[9] + 2.49687725014238*U[18]*U[8] - 4.63705775026441*U[19]*U[7] + 4.63705775026441*U[20]*U[6] + 0.00194123551627786*U[21] - 0.0231453954121832*U[23] - 1.89187704652139e-5*U[29]**3*U[30] + 3.4053786837385e-5*U[29]**3*U[32] - 0.00264469900447931*U[30] + 0.00476045820806276*U[32] + 0.0142857142857143*U[3]
    U_out[15] = -1.16886956037576*U[0]*U[16] - 0.00141372192628079*U[10]**3*U[15] + 0.422511733532696*U[11]*U[5] - 0.612715105162524*U[12]*U[7] - 1.38291034440645*U[13]*U[3] + 0.612715105162524*U[13]*U[6] + 1.79985224341047*U[14]*U[2] - 0.0377373135508224*U[15] + 0.0207014663040147*U[16] - 1.87294455066922*U[17]*U[2] + 1.87294455066922*U[18]*U[1] + 0.00598141266899433*U[22] - 1.87500693937487e-5*U[29]**3*U[31] - 0.00262111588862201*U[31] + 0.0181325685149777*U[4]
    U_out[16] = 1.16886956037576*U[0]*U[15] - 0.00141372192628079*U[10]**3*U[16] - 0.422511733532696*U[11]*U[4] + 1.38291034440645*U[12]*U[3] + 0.612715105162524*U[12]*U[6] + 0.612715105162524*U[13]*U[7] - 1.79985224341047*U[14]*U[1] - 0.0207014663040147*U[15] - 0.0377373135508224*U[16] - 1.87294455066922*U[17]*U[1] - 1.87294455066922*U[18]*U[2] - 0.0181325685149777*U[26] + 4.41788101962747e-5*U[29]**3*U[35] + 0.0061758588149268*U[35] + 0.0181325685149777*U[5]
    U_out[17] = -2.94535914360826*U[0]*U[18] - 0.00124894690446766*U[10]**3*U[17] + 0.569389237785845*U[11]*U[7] - 0.768581081081081*U[12]*U[5] - 0.768581081081081*U[13]*U[4] + 0.911022780457353*U[14]*U[9] + 1.42736486486486*U[15]*U[2] + 1.42736486486486*U[16]*U[1] - 0.0414976604511488*U[17] + 0.0365772529628368*U[18] - 4.71257462977322*U[20]*U[3] + 0.000339936796618951*U[21] - 3.31293455842248e-6*U[29]**3*U[30] - 0.000463122841131525*U[30] + 0.0218468468468468*U[6]
    U_out[18] = 2.94535914360826*U[0]*U[17] - 0.00124894690446766*U[10]**3*U[18] - 0.569389237785845*U[11]*U[6] + 0.768581081081081*U[12]*U[4] - 0.768581081081081*U[13]*U[5] - 0.911022780457353*U[14]*U[8] - 1.42736486486486*U[15]*U[1] + 1.42736486486486*U[16]*U[2] - 0.0365772529628368*U[17] - 0.0414976604511488*U[18] + 4.71257462977322*U[19]*U[3] + 0.0218468468468468*U[7]
    U_out[19] = -2.37660377951895*U[0]*U[20] - 0.00106846324775268*U[10]**3*U[19] + 0.0288656329496229*U[11]*U[9] + 1.50101291338039*U[14]*U[7] - 3.30992591155675*U[18]*U[3] - 0.0456164956460695*U[19] + 0.031291522765895*U[20] + 0.000904126828290182*U[22] - 2.83418678986432e-6*U[29]**3*U[31] - 0.000396197575072056*U[31] + 0.0259152215799615*U[8]
    U_out[20] = 2.37660377951895*U[0]*U[19] - 0.00106846324775268*U[10]**3*U[20] - 0.0288656329496229*U[11]*U[8] - 1.50101291338039*U[14]*U[6] + 3.30992591155675*U[17]*U[3] - 0.031291522765895*U[19] - 0.0456164956460695*U[20] + 0.0259152215799615*U[9]
    U_out[21] = 6.35094981132412e-8*U[12] + 8.50239961075093e-8*U[14] + 3.66419111790894e-8*U[17] - 6.35094981132412e-8*U[1] - 1.58263150747531e-7*U[21] - 0.000969739516006511*U[22]*U[25] - 0.00116937805032627*U[22]*U[27] - 0.00278561057700379*U[23]*U[26] - 0.00308506837848344*U[23]*U[28] - 0.00534779391583196*U[24]*U[27] - 7.91389255433123e-5*U[25] - 8.50239961075093e-8*U[3] - 3.66419111790894e-8*U[6]
    U_out[22] = 4.2438907259419e-8*U[0] - 4.2438907259419e-8*U[11] + 1.34106967724862e-7*U[15] + 5.07202450868017e-8*U[19] - 0.000708243330589963*U[21]*U[25] + 0.00172613939940631*U[21]*U[27] - 4.91188341087484e-7*U[22] - 0.00626988589506499*U[23]*U[25] - 0.0119940379417069*U[24]*U[26] - 7.9002862144845e-5*U[26] - 1.34106967724862e-7*U[4] - 5.07202450868017e-8*U[8]
    U_out[23] = -1.52343526260616e-7*U[14] - 0.00317032731211854*U[21]*U[26] + 0.0060425658523595*U[21]*U[28] + 0.00160884360522711*U[22]*U[25] - 1.04352693785129e-6*U[23] - 0.0178532887343688*U[24]*U[25] - 7.87771265478973e-5*U[27] + 1.52343526260616e-7*U[3]
    U_out[24] = 1.68596160851496e-8*U[0] - 1.68596160851496e-8*U[11] - 0.00857174744319731*U[21]*U[27] - 0.00140681174654471*U[22]*U[26] + 0.00871788567853858*U[23]*U[25] - 1.8115193420227e-6*U[24] - 7.84632552652771e-5*U[28]
    U_out[25] = -1.49531921957748e-7*U[13] + 0.00167798163428125*U[21]*U[22] + 7.9081381622411e-5*U[21] + 0.0046610600952257*U[22]*U[23] + 0.00913567778664237*U[23]*U[24] - 2.99063843915495e-7*U[25] + 1.49531921957748e-7*U[2]
    U_out[26] = -3.15752734584711e-7*U[16] + 0.0059559067658186*U[21]*U[23] + 0.0134007902230919*U[22]*U[24] + 7.89455158524528e-5*U[22] - 6.31505469169422e-7*U[26] + 3.15752734584711e-7*U[5]
    U_out[27] = -0.0005567719890759*U[21]*U[22] + 0.0139192997268975*U[21]*U[24] + 7.8720107381223e-5*U[23] - 1.18304364206348e-6*U[27]
    U_out[28] = -0.00295762797162433*U[21]*U[23] + 7.84066893926385e-5*U[24] - 1.94992690884148e-6*U[28]
    U_out[29] = 2.7726621279182e-5*U[10]**4 + 0.000141323097779036*U[10]**3*U[12] + 5.0894196312133e-5*U[10]**3*U[14] + 2.82646195558073e-5*U[10]**3*U[17] + 0.00193798449615121*U[10] + 0.00246948564053933*U[12] + 0.000889327285869774*U[14] + 0.000493897128107866*U[17] + 1.30103637962932*U[21]*U[35] - 0.650518189814661*U[21]*U[37] - 1.30103637961282*U[22]*U[34] + 0.975777284716489*U[22]*U[36] - 3.90310913886596*U[23]*U[35] + 1.95155456943298*U[23]*U[37] - 2.27681366435131*U[24]*U[34] - 4.87888642358245*U[24]*U[36] + 1.30103637961282*U[25]*U[31] + 2.27681366435131*U[25]*U[33] - 1.30103637962932*U[26]*U[30] + 3.90310913886596*U[26]*U[32] - 0.975777284716489*U[27]*U[31] + 4.87888642358245*U[27]*U[33] + 0.650518189814661*U[28]*U[30] - 1.95155456943298*U[28]*U[32] - 2.28014977624852e-6*U[29]**4 + 8.470329472543e-22*U[29]**3*U[30] + 2.5410988417629e-21*U[29]**3*U[32] - 0.000968992248062015*U[29] + 0.000180438142739942
    U_out[30] = -3.3881317890172e-21*U[10]**4 - 0.000161622362549857*U[10]**3*U[12] - 8.36313275290456e-5*U[10]**3*U[14] - 3.23244725099714e-5*U[10]**3*U[17] - 3.96655364803822e-16*U[10] - 0.00282419583055875*U[12] - 0.00146137726726263*U[14] - 0.000564839166111749*U[17] - 1.05458036756628*U[21]*U[35] + 0.52729018378314*U[21]*U[37] + 2.02958036756658*U[22]*U[34] - 1.11593527567481*U[22]*U[36] + 4.46374110269924*U[23]*U[35] - 2.23187055134962*U[23]*U[37] + 1.84551564324099*U[24]*U[34] + 5.57967637837405*U[24]*U[36] - 2.02958036756658*U[25]*U[31] - 1.84551564324099*U[25]*U[33] + 1.05458036756628*U[26]*U[30] - 4.46374110269924*U[26]*U[32] + 1.11593527567481*U[27]*U[31] - 5.57967637837405*U[27]*U[33] - 0.52729018378314*U[28]*U[30] + 2.23187055134962*U[28]*U[32] - 8.470329472543e-22*U[29]**4 - 9.12059910499407e-6*U[29]**3*U[30] - 2.5410988417629e-21*U[29]**3*U[32] - 0.000968992248062015*U[30] - 3.68628738645072e-17
    U_out[31] = 8.47560916841583e-5*U[10]**3*U[11] - 4.70701742060666e-5*U[10]**3*U[15] - 9.41403484121332e-6*U[10]**3*U[19] + 0.00148103144250841*U[11] - 0.000822506165849588*U[15] - 0.000164501233169918*U[19] - 0.975*U[21]*U[34] + 0.325*U[21]*U[36] + 1.625*U[23]*U[34] + 1.95*U[24]*U[35] + 0.975*U[25]*U[30] - 1.625*U[25]*U[32] - 1.95*U[26]*U[33] - 0.325*U[27]*U[30] - 9.12059910499408e-6*U[29]**3*U[31] - 0.000968992248062015*U[31] + 0.000137893034416115
    U_out[32] = -3.3881317890172e-21*U[10]**4 - 3.81840627812635e-5*U[10]**3*U[12] + 6.25293886200869e-5*U[10]**3*U[14] - 7.63681255625269e-6*U[10]**3*U[17] - 2.16840434497101e-19*U[10] - 0.0006672298882363*U[12] + 0.00109264111625484*U[14] - 0.00013344597764726*U[17] - 1.65152678918876*U[21]*U[35] + 0.82576339459438*U[21]*U[37] - 1.27347321081124*U[22]*U[34] - 0.26364509189157*U[22]*U[36] + 1.05458036756628*U[23]*U[35] - 0.527290183783139*U[23]*U[37] + 2.89017188108033*U[24]*U[34] + 1.31822545945785*U[24]*U[36] + 1.27347321081124*U[25]*U[31] - 2.89017188108033*U[25]*U[33] + 1.65152678918876*U[26]*U[30] - 1.05458036756628*U[26]*U[32] + 0.26364509189157*U[27]*U[31] - 1.31822545945785*U[27]*U[33] - 0.82576339459438*U[28]*U[30] + 0.527290183783139*U[28]*U[32] + 2.11758236813575e-22*U[29]**4 - 9.12059910499408e-6*U[29]**3*U[32] - 0.000968992248062015*U[32] + 2.71050543121376e-20
    U_out[33] = 3.39024366736633e-5*U[10]**3*U[11] + 0.000592412577003366*U[11] - 1.625*U[21]*U[36] - 1.95*U[22]*U[35] - 2.275*U[23]*U[34] + 2.275*U[25]*U[32] + 1.95*U[26]*U[31] + 1.625*U[27]*U[30] - 9.12059910499408e-6*U[29]**3*U[33] - 0.000968992248062015*U[33] + 5.51572137664459e-5
    U_out[34] = 0.000110906485116728*U[10]**3*U[13] + 0.00193798449612403*U[13] + 0.975*U[21]*U[31] - 0.975*U[22]*U[30] + 1.625*U[22]*U[32] - 1.625*U[23]*U[31] + 2.275*U[23]*U[33] - 2.275*U[24]*U[32] - 9.12059910499408e-6*U[29]**3*U[34] - 0.000968992248062015*U[34]
    U_out[35] = 0.000110906485116728*U[10]**3*U[16] + 0.00193798449612403*U[16] + 1.3*U[21]*U[32] + 1.95*U[22]*U[33] - 1.3*U[23]*U[30] - 1.95*U[24]*U[31] - 9.12059910499408e-6*U[29]**3*U[35] - 0.000968992248062015*U[35]
    U_out[36] = -0.325*U[21]*U[31] + 1.625*U[21]*U[33] + 0.325*U[22]*U[30] - 1.625*U[24]*U[30] - 9.12059910499408e-6*U[29]**3*U[36] - 0.000968992248062015*U[36]
    U_out[37] = -0.65*U[21]*U[32] + 0.65*U[23]*U[30] - 9.12059910499408e-6*U[29]**3*U[37] - 0.000968992248062015*U[37]
    return U_out

In [ ]:
from qgs.integrators.integrator import RungeKuttaIntegrator, RungeKuttaTglsIntegrator
import matplotlib.pyplot as plt

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

In [ ]:
%%time
ic = np.random.rand(model_parameters.ndim)*0.1
integrator.integrate(0., 1000000., 0.1, ic=ic, write_steps=0)
time, ic = integrator.get_trajectories()

In [ ]:
ic = np.array([
    4.64467907e-02, -2.25054437e-02, -3.51762104e-02,  4.65057475e-03,
   -5.08842464e-03,  1.92789965e-02,  3.47849539e-03,  1.86038254e-02,
   -2.99742925e-03,  8.85862363e-03,  1.53577181e+00,  4.29584574e-02,
   -1.88143295e-02, -1.15114614e-02,  1.24843108e-03, -3.61467108e-03,
    2.81459020e-03,  5.59630451e-03,  3.56804517e-03, -1.52654934e-03,
    3.09046486e-03,  1.18185262e-06,  6.76756261e-04,  4.21981618e-06,
    1.06290914e-05, -1.64056994e-05,  3.40932989e-05,  1.90943692e-05,
    1.08189600e-05,  3.16661484e+00, -3.06749318e-04,  1.82036792e-01,
   -1.33924039e-04,  9.77967977e-03, -5.58853689e-03,  2.33218135e-02,
    3.45971396e-05, -5.88894363e-05
])

In [ ]:
%%time
integrator.integrate(0., 1000000., 0.1, ic=ic, write_steps=5)
reference_time, reference_traj = integrator.get_trajectories()

In [ ]:
varx = 22
vary = 31
plt.figure(figsize=(10, 8))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.07, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');